In [5]:
import getpass
import os


os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
import vertexai

class StreamingChatMemory:
    def __init__(self, model="gemini-1.5-flash-001"):
        # Initialize Vertex AI
        vertexai.init(project='gen-lang-client-0811883729', location='us-central1')
        
        # Create Language Model with streaming
        self.llm = ChatVertexAI(
            model=model,
            streaming=True,
            temperature=0.7
        )
        
        # Initialize memory
        self.memory = ConversationBufferMemory(return_messages=True)
    
    def stream_chat(self, user_input):
        # Prepare conversation history
        chat_history = self.memory.chat_memory.messages
        
        # Combine chat history with new input
        messages = chat_history + [HumanMessage(content=user_input)]
        
        # Stream response
        print("AI: ", end='', flush=True)
        full_response = ""
        
        for chunk in self.llm.stream(messages):
            # Extract content from AIMessageChunk
            text_chunk = chunk.content if hasattr(chunk, "content") else str(chunk)
            print(text_chunk, end='', flush=True)  # Print chunk to console
            full_response += text_chunk  # Concatenate chunk to the full response
        
        print()  # New line after response
        
        # Save context to memory
        self.memory.save_context(
            {"input": user_input}, 
            {"output": full_response}
        )
        
        return full_response

    
    def interactive_chat(self):
        print("Streaming Chat Started. Type 'exit' to end.")
        
        while True:
            # Get user input
            user_input = input("You: ")
            print(f"Me : " + user_input)
            # Exit condition
            if user_input.lower() in ['exit', 'quit', 'bye']:
                break
            
            # Stream and memorize response
            self.stream_chat(user_input)
    
    # Unneccesary function generated by a Gen AI
    def print_history(self):
        # Print conversation history
        print("\n--- Conversation History ---")
        for msg in self.memory.chat_memory.messages:
            if isinstance(msg, HumanMessage):
                print(f"You: {msg.content}")
            elif isinstance(msg, AIMessage):
                print(f"AI: {msg.content}")

# Run the chat
def main():
    chat = StreamingChatMemory()
    
    try:
        chat.interactive_chat()
    finally:
        # Print conversation history at the end
        # chat.print_history()
        print("Conversation Ended")

if __name__ == "__main__":
    main()